In [ ]:
import numpy as np
import pyafn

print("=" * 60)
print("EXAMPLE 1: Single Room with 4 Openings")
print("=" * 60)

flowParams = pyafn.createFlowParams(
    C_d=[1, 1, 1, 1],
    A=[1, 1, 1, 1],
    p_w=[1, 3, 1, -3],
    z=[3, 3, 6, 3],
    delT=[-3, 0, 3, 0],
    q=[1, 2, -1, -2],
    rooms=[[1], [1], [1], [1]],
    hr=3
)

p_0s = np.array([1])

print(f"\nGiven indoor pressure p0 = {p_0s[0]} Pa")
print("\nFlow rates through each opening (m³/s):")
print(pyafn.flowField(p_0s, flowParams))
print(f"\nContinuity residual (sum of q²): {pyafn.qObjective(p_0s, flowParams):.4f}")
print("\nCalculated discharge coefficients:")
print(pyafn.getC(p_0s, flowParams))

In [ ]:
import matplotlib.pyplot as plt

print("\nPlotting objective function vs. indoor pressure...")

plt.figure(figsize=(10, 6))
p0s = np.linspace(min(pyafn.getWindBuoyantP(flowParams)), 
                  max(pyafn.getWindBuoyantP(flowParams)), 100)
qs = [pyafn.qObjective(np.array([p0]), flowParams) for p0 in p0s]
plt.plot(p0s, qs, linewidth=2)
plt.xlabel("Indoor Pressure p0 (Pa)", fontsize=12)
plt.ylabel("Objective Function $\sum q^2$ (m⁶/s²)", fontsize=12)
plt.title("Continuity Residual vs. Indoor Pressure", fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
print("\n" + "=" * 60)
print("Finding Optimal Indoor Pressure")
print("=" * 60)

optResults = pyafn.findOptimalP0(flowParams)
p_0s = optResults.x

print(f"\nOptimization Status: {optResults.message}")
print(f"Success: {optResults.success}")
print(f"\nOptimal indoor pressure: {p_0s[0]:.4f} Pa")
print(f"Final objective value: {optResults.fun:.6e}")

print("\nFlow rates at optimal pressure (m³/s):")
flows = pyafn.flowField(p_0s, flowParams)
print(flows)
print(f"Sum of flows: {np.sum(flows):.6e} (should be ~0 for mass balance)")

print("\nContinuity check:")
print(f"Objective (sum of q²): {pyafn.qObjective(p_0s, flowParams):.6e}")

print("\nCalculated discharge coefficients:")
print(pyafn.getC(p_0s, flowParams))

In [ ]:
print("\n" + "=" * 60)
print("Joint Optimization of Pressures and Discharge Coefficients")
print("=" * 60)
print("\nThis optimizes both p0 and C_d to match target flows")
print("while encouraging uniform discharge coefficients.\n")

# Jointly optimize pressures and discharge coefficients
flowParams = pyafn.createFlowParams(
    C_d=[1, 1, 1, 1],
    A=[1, 1, 1, 1],
    p_w=[1, 3, 1, -3],
    z=[3, 3, 6, 3],
    delT=[-3, 0, 3, 0],
    q=[1, 2, -1, -2],
    rooms=[[1], [1], [1], [1]],
    hr=3
)
p_0s = np.array([1])

optResults = pyafn.findOptimalP0AndC(flowParams, weight=1e-1, disp=True)
n_rooms = flowParams["rooms"].shape[1] 
p_0s = optResults.x[0:n_rooms]
flowParams["C_d"] = optResults.x[n_rooms:]

print(f"\nOptimization Status: {optResults.message}")
print(f"Success: {optResults.success}")
print(f"\nOptimal indoor pressure: {p_0s}")
print(f"Optimal discharge coefficients: {flowParams['C_d']}")

print("\nPredicted flow rates (m³/s):")
print(pyafn.flowField(p_0s, flowParams))

print("\nBack-calculated discharge coefficients:")
print(pyafn.getC(p_0s, flowParams))

In [ ]:
print("\n" + "=" * 60)
print("EXAMPLE 2: Two Rooms with 5 Openings (1D Temperature)")
print("=" * 60)

flowParams = pyafn.createFlowParams(
    C_d=[1, 1, 1, 1, 10],
    A=[1, 1, 1, 1, 3],
    p_w=[1, 3, 1, -3, 0],
    z=[3, 3, 6, 3, 0],
    delT=[-3, 0, 3, 0, 0],
    q=[2, 2, -1, -3, -4],
    rooms=[[1, 0], [1, 0], [0, 1], [0, 1], [1, -1]],
    hr=3
)
p_0s = np.array([1, -1])

print(f"\nGiven indoor pressures: Room 1 = {p_0s[0]} Pa, Room 2 = {p_0s[1]} Pa")
print("\nFlow rates through each opening (m³/s):")
flows = pyafn.flowField(p_0s, flowParams)
print(flows)

print(f"\nContinuity residual: {pyafn.qObjective(p_0s, flowParams):.4f}")

print("\nCalculated discharge coefficients:")
print(pyafn.getC(p_0s, flowParams))

In [ ]:
print("\n" + "=" * 60)
print("EXAMPLE 3: Two Rooms with 5 Openings (2 Temperature Layers)")
print("=" * 60)
print("Temperature varies with height in each room")

flowParams = pyafn.createFlowParams(
    C_d=[1, 1, 1, 1, 10],
    A=[1, 1, 1, 1, 3],
    p_w=[1, 3, 1, -3, 0],
    z=[1.5, 4, 3, 1.5, 0],
    delT=[[-3, -2], [0, 1], [2, 3], [0, 0], [0, 0]],
    q=[2, 2, -1, -3, -4],
    rooms=[[1, 0], [1, 0], [0, 1], [0, 1], [1, -1]],
    hr=3
)
p_0s = np.array([1, -1])

print(f"\nGiven indoor pressures: Room 1 = {p_0s[0]} Pa, Room 2 = {p_0s[1]} Pa")
print("\nFlow rates through each opening (m³/s):")
flows = pyafn.flowField(p_0s, flowParams)
for i, flow in enumerate(flows):
    print(f"  Opening {i}: {flow:7.3f}")

print(f"\nContinuity residual: {pyafn.qObjective(p_0s, flowParams):.4f}")

print("\nCalculated discharge coefficients:")
Cd_calc = pyafn.getC(p_0s, flowParams)
for i, cd in enumerate(Cd_calc):
    print(f"  Opening {i}: {cd:.4f}")

In [ ]:
print("\n" + "=" * 60)
print("Optimizing Pressures for Example 3")
print("=" * 60)

optResults = pyafn.findOptimalP0(flowParams)
p_0s = optResults.x

print(f"\nOptimization Status: {optResults.message}")
print(f"Success: {optResults.success}")

print(f"\nOptimal indoor pressures:")
print(f"  Room 1: {p_0s[0]:.4f} Pa")
print(f"  Room 2: {p_0s[1]:.4f} Pa")

print(f"\nFinal objective value: {optResults.fun:.6e}")

print("\nFlow rates at optimal pressures (m³/s):")
flows = pyafn.flowField(p_0s, flowParams)
for i, flow in enumerate(flows):
    print(f"  Opening {i}: {flow:7.3f}")

# Check mass balance for each room
rooms = flowParams["rooms"]
room_flows = np.matmul(rooms.T, flows)
print("\nMass balance check (flow into each room):")
for i, room_flow in enumerate(room_flows):
    print(f"  Room {i+1}: {room_flow:.6e} m³/s (should be ~0)")

print(f"\nContinuity residual: {pyafn.qObjective(p_0s, flowParams):.6e}")

print("\nCalculated discharge coefficients:")
Cd_calc = pyafn.getC(p_0s, flowParams)
for i, cd in enumerate(Cd_calc):
    print(f"  Opening {i}: {cd:.4f}")

In [ ]:
print("\n" + "=" * 60)
print("Joint Optimization of Pressures and Discharge Coefficients")
print("(Two-Room Case)")
print("=" * 60)
print("\nMatching target flows while encouraging uniform C_d values\n")

# Jointly optimize pressures and discharge coefficients (two-room case)
flowParams = pyafn.createFlowParams(
    C_d=[1, 1, 1, 1, 10],
    A=[1, 1, 1, 1, 3],
    p_w=[1, 3, 1, -3, 0],
    z=[1.5, 4, 3, 1.5, 0],
    delT=[[-3, -2], [0, 1], [2, 3], [0, 0], [0, 0]],
    q=[2, 2, -1, -3, -4],
    rooms=[[1, 0], [1, 0], [0, 1], [0, 1], [1, -1]],
    hr=3
)
p_0s = np.array([1, -1])

optResults = pyafn.findOptimalP0AndC(flowParams, weight=1e-1)
n_rooms = flowParams["rooms"].shape[1]
p_0s = optResults.x[0:n_rooms]
flowParams["C_d"] = optResults.x[n_rooms:]

print(f"Optimization Status: {optResults.message}")
print(f"Success: {optResults.success}")

print(f"\nOptimal indoor pressures:")
for i, p in enumerate(p_0s):
    print(f"  Room {i+1}: {p:.4f} Pa")

print(f"\nOptimal discharge coefficients:")
for i, cd in enumerate(flowParams["C_d"]):
    print(f"  Opening {i}: {cd:.4f}")

print("\nPredicted flow rates (m³/s):")
flows = pyafn.flowField(p_0s, flowParams)
for i, flow in enumerate(flows):
    print(f"  Opening {i}: {flow:7.3f}")

print("\nBack-calculated C_d (should match optimal C_d):")
Cd_calc = pyafn.getC(p_0s, flowParams)
for i, cd in enumerate(Cd_calc):
    print(f"  Opening {i}: {cd:.4f}")